In [2]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
from scipy.stats import trim_mean
from sklearn.metrics import mean_squared_error

sys.path.append('..')
from higgs_inference import settings
from higgs_inference.various.utils import format_number

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
result_dir = '../results/'

# TablePrinter class

In [4]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Skip entirely empty lines
            if not np.any(np.isfinite(line[2:])):
                continue
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j])) + ')'
                    else:
                        text += format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        self.table += self.indent + self.midrule + self.end_line
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            bracket = False
            try:
                value = fn(filename, folder)
            except (IOError, ValueError):
                #print('File', filename, 'in folder', folder, 'not found')
                value = np.nan
                
            if isinstance(value, (list, tuple)):
                value, bracket = value
                
            line.append(value)
            brackets.append(bracket)

        self.block_entries.append(line)
        self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [5]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.sqrt(np.median(errors))

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.median(errors)

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return np.sqrt(trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return np.sqrt(trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

# Main tables

In [18]:
labels = [r'AFC (density est.\ in $\boldx$)',
          'carl (PbP, raw)',
          'carl (PbP, cal.)',
          'carl (param., raw)',
          'carl (param., cal.)',
          'carl + score (param., raw)',
          'carl + score (param., cal.)',
          'SM score regression',
          'Ratio regression (PbP, raw)',
          'Ratio regression (param., raw)',
          r'Ratio regr.\ + score (param., raw)']

folders = (['afc']
          + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['score_regression']
          + ['point_by_point'] + ['parameterized'] * 2)

filenames = ['afc',
             'carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'combined',
             'combined_calibrated',
             'scoreregression',
             'regression',
             'regression',
             'combinedregression']


def show_main_table(algorithm_begin=0, algorithm_end=None):
    
    table = TablePrinter([mse_log_r_nottrained, trimmed_mse_log_r_nottrained,
                          mse_expected_log_r],
                          precisions=[2,2,1])

    for i, (label, filename, folder) in enumerate(
        zip(labels[algorithm_begin:algorithm_end], filenames[algorithm_begin:algorithm_end], folders[algorithm_begin:algorithm_end])):

        if i > 0:
            table.new_block()

        if folder == 'point_by_point':
            table.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
            table.add(label, 'PbP (2)', filename, 'point_by_point')
            table.add('', 'PbP (3)', filename + '_deep', 'point_by_point')

        elif folder == 'afc':
            table.add(label, '2d, $\epsilon = 0.55$ (2)', filename + '_2d', 'afc')
            table.add(label, '2d, $\epsilon = 0.2$ (2)', filename + '_2d_epsilon_0.20', 'afc')

            table.add(label, '5d, $\epsilon = 0.2$ (2)', filename + '_5d', 'afc')
            table.add(label, '5d, $\epsilon = 0.1$ (2)', filename + '_5d_epsilon_0.10', 'afc')

        elif folder == 'score_regression':
            table.add(label, r'Fixed density est. on $t$  (2)', filename + '_score', 'score_regression')
            table.add('', r'Dyn. dens. est. on $t$  (2)', filename + '_rotatedscore', 'score_regression')
            table.add('', r'Dens. est. on $t \cdot \theta$  (2)', filename + '_scoretheta', 'score_regression')

        else:
            table.add(label, 'Baseline (1)', filename + '_shallow')
            table.add(label, 'Baseline (2)', filename)
            table.add('', 'Baseline (3)', filename + '_deep')
            
            
            table.add(label, r'Baseline, $\alpha = 0.01$ (2)', filename + '_alpha_0.01')
            table.add(label, r'Baseline, $\alpha = 0.03$ (2)', filename + '_alpha_0.03')
            table.add(label, r'Baseline, $\alpha = 0.3$ (2)', filename + '_alpha_0.3')
            table.add(label, r'Baseline, $\alpha = 1$ (2)', filename + '_alpha_1')
            table.add(label, r'Baseline, $\alpha = 30$ (2)', filename + '_alpha_30')
            table.add(label, r'Baseline, $\alpha = 300$ (2)', filename + '_alpha_300')
            table.add(label, r'Baseline, $\alpha = 1000$ (2)', filename + '_alpha_1000')
            
            table.add('', 'Baseline, large batch (2)', filename + '_largebatch')
            table.add('', 'Baseline, small batch (2)', filename + '_smallbatch')
            table.add('', 'Baseline, LR decay (2)', filename + '_lrdecay')
            table.add('', 'Baseline, LR decay, large batch (2)', filename + '_lrdecay_largebatch')
            table.add('', 'Baseline, LR decay, small batch (2)', filename + '_lrdecay_largebatch')
            table.add('', 'Baseline, small LR (2)', filename + '_slowlearning')
            table.add('', 'Baseline, small LR, large batch (2)', filename + '_slowlearning_largebatch')
            table.add('', 'Baseline, small LR, small batch (2)', filename + '_slowlearning_smallbatch')
            table.add('', 'Baseline, small LR, LR decay (2)', filename + '_slowlearning_lrdecay')
            table.add('', 'Baseline, small LR, LR decay, large batch (2)', filename + '_slowlearning_lrdecay_largebatch')
            table.add('', 'Baseline, small LR, LR decay, small batch (2)', filename + '_slowlearning_lrdecay_smallbatch')
            table.add('', 'Baseline, large LR (2)', filename + '_fastlearning')
            table.add('', 'Baseline, large LR, large batch (2)', filename + '_fastlearning_largebatch')
            table.add('', 'Baseline, large LR, small batch (2)', filename + '_fastlearning_smallbatch')
            table.add('', 'Baseline, large LR, LR decay (2)', filename + '_fastlearning_lrdecay')
            table.add('', 'Baseline, large LR, LR decay, large batch (2)', filename + '_fastlearning_lrdecay_largebatch')
            table.add('', 'Baseline, large LR, LR decay, small batch (2)', filename + '_fastlearning_lrdecay_smallbatch')
            
            table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
            table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
            table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')
            
            table.add('', 'Aware, baseline (1)', filename + '_aware_shallow')
            table.add('', 'Aware, baseline (2)', filename + '_aware')
            table.add('', 'Aware, baseline (3)', filename + '_aware_deep')
            table.add('', 'Aware, baseline, SM dev. (2)', filename + '_aware_factorsm')
            table.add('', 'Aware, baseline, small LR (2)', filename + '_aware_slowlearning')
            table.add('', 'Aware, baseline, small LR, SM dev. (2)', filename + '_aware_factorsm_slowlearning')
            table.add('', 'Aware, basis (2)', filename + '_aware_basis')

    print(table.print())

In [19]:
show_main_table(0,5)

   AFC (density est.\ in $\boldx$) & 2d, $\epsilon = 0.55$ (2) & \emph{0.46} & \emph{0.16} & \emph{5.2}\\
   AFC (density est.\ in $\boldx$) & 5d, $\epsilon = 0.2$ (2) & 0.57 & 0.21 & 5.7\\
   \midrule
   \midrule
   \midrule
   carl (param., raw) & Baseline (2) & 0.33 & 0.19 & 12.8\\
    & Baseline, large batch (2) & 0.30 & 0.14 & 5.8\\
    & Baseline, LR decay, large batch (2) & \emph{0.21} & 0.07 & 1.6\\
    & Baseline, LR decay, small batch (2) & 0.21 & 0.07 & 1.6\\
    & Baseline, small LR, large batch (2) & 0.25 & \emph{0.07} & \emph{1.5}\\
    & Baseline, large LR (2) & 0.53 & 0.27 & 6.3\\
    & Baseline, large LR, large batch (2) & 0.72 & 0.55 & 35.3\\
    & Baseline, large LR, LR decay, large batch (2) & 0.39 & 0.17 & 10.9\\
    & Random $\boldtheta$ (2) & 0.31 & 0.16 & 6.6\\
   \midrule
   carl (param., cal.) & Baseline (2) & 0.29 & 0.09 & 1.0\\
    & Baseline, large batch (2) & 0.27 & 0.10 & 1.3\\
    & Baseline, LR decay, large batch (2) & \emph{0.21} & 0.06 & \emph{0.6}\\


In [16]:
show_main_table(5,8)

   carl + score (param., raw) & Baseline (2) & 0.25 & 0.12 & 4.2\\
    & Random $\boldtheta$ (2) & \emph{0.23} & \emph{0.09} & \emph{1.3}\\
   \midrule
   carl + score (param., cal.) & Baseline (2) & 0.23 & 0.08 & 1.0\\
    & Random $\boldtheta$ (2) & \emph{0.21} & \emph{0.08} & \emph{0.9}\\
   \midrule
   SM score regression & Fixed density est. on $t$  (2) & 0.19 & 0.06 & \emph{1.7}\\
    & Dyn. dens. est. on $t$  (2) & 0.19 & 0.06 & 1.8\\
    & Dens. est. on $t \cdot \theta$  (2) & \emph{0.19} & \emph{0.05} & 3.1\\



In [17]:
show_main_table(8,None)

   \midrule
   Ratio regression (param., raw) & Baseline (2) & 0.22 & 0.16 & 8.2\\
    & Baseline, large batch (2) & 0.19 & 0.12 & 1.4\\
    & Baseline, small batch (2) & 0.19 & 0.12 & 2.9\\
    & Baseline, LR decay (2) & \emph{0.12} & \emph{0.07} & \emph{0.9}\\
    & Baseline, LR decay, large batch (2) & 0.15 & 0.10 & 1.1\\
    & Baseline, LR decay, small batch (2) & 0.15 & 0.10 & 1.1\\
    & Baseline, small LR (2) & 0.20 & 0.14 & 7.3\\
    & Baseline, small LR, large batch (2) & 0.21 & 0.15 & 4.3\\
    & Baseline, small LR, LR decay, large batch (2) & 0.34 & 0.25 & 3.6\\
    & Baseline, large LR (2) & 0.22 & 0.14 & 3.4\\
    & Baseline, large LR, large batch (2) & 0.21 & 0.14 & 2.4\\
    & Baseline, large LR, small batch (2) & 0.20 & 0.12 & 4.2\\
    & Baseline, large LR, LR decay (2) & 0.36 & 0.24 & \\
    & Baseline, large LR, LR decay, large batch (2) & 0.37 & 0.25 & 8.0\\
    & Random $\boldtheta$ (2) & 0.22 & 0.15 & 5.2\\
   \midrule
   Ratio regr.\ + score (param., raw) & Basel

# Score accuracy table

In [10]:
labels = ['carl',
          #'Score',
          'carl + score',
          'Ratio regression',
          'Ratio regr.\ + score']

filenames = ['carl',
             #'score',
             'combined',
             'regression',
             'combinedregression']

table = TablePrinter([mse_score_nottrained, trimmed_mse_score_nottrained],
                      precisions=[2,2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table.new_block()
        
    table.add(label, 'Baseline (1)', filename + '_shallow')
    table.add(label, 'Baseline (2)', filename)
    table.add('', 'Baseline (3)', filename + '_deep')
    table.add('', 'Baseline, large batch (2)', filename + '_largebatch')
    table.add('', 'Baseline, small batch (2)', filename + '_smallbatch')
    table.add('', 'Baseline, small LR (2)', filename + '_slowlearning')
    table.add('', 'Baseline, small LR, large batch (2)', filename + '_slowlearning_largebatch')
    table.add('', 'Baseline, small LR, small batch (2)', filename + '_slowlearning_smallbatch')
    table.add('', 'Baseline, large LR (2)', filename + '_slowlearning')
    table.add('', 'Baseline, large LR, large batch (2)', filename + '_slowlearning_largebatch')
    table.add('', 'Baseline, large LR, small batch (2)', filename + '_slowlearning_smallbatch')

    table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
    table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
    table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

    table.add('', 'Aware, baseline (1)', filename + '_aware_shallow')
    table.add('', 'Aware, baseline (2)', filename + '_aware')
    table.add('', 'Aware, baseline (3)', filename + '_aware_deep')
    table.add('', 'Aware, baseline, SM dev. (2)', filename + '_aware_factorsm')
    table.add('', 'Aware, baseline, small LR (2)', filename + '_aware_slowlearning')
    table.add('', 'Aware, baseline, small LR, SM dev. (2)', filename + '_aware_factorsm_slowlearning')
    table.add('', 'Aware, basis (2)', filename + '_aware_basis')

print(table.print())

   carl & Baseline (2) & 27.55 & 0.66\\
    & Baseline, large batch (2) & 27.54 & 0.46\\
    & Baseline, small LR, large batch (2) & 27.53 & \emph{0.24}\\
    & Baseline, large LR, large batch (2) & 27.53 & 0.24\\
    & Random $\boldtheta$ (2) & \emph{27.52} & 0.66\\
   \midrule
   carl + score & Baseline (2) & 27.51 & 0.22\\
    & Random $\boldtheta$ (2) & \emph{27.50} & \emph{0.19}\\
   \midrule
   Ratio regression & Baseline (2) & 27.47 & \emph{0.52}\\
    & Baseline, large batch (2) & 27.54 & 0.78\\
    & Baseline, small batch (2) & 27.52 & 0.71\\
    & Baseline, small LR (2) & 27.47 & 0.76\\
    & Baseline, small LR, large batch (2) & \emph{27.45} & 0.83\\
    & Baseline, large LR (2) & 27.47 & 0.76\\
    & Baseline, large LR, large batch (2) & 27.45 & 0.83\\
    & Random $\boldtheta$ (2) & 27.55 & 0.86\\
   \midrule
   Ratio regr.\ + score & Baseline (2) & \emph{27.49} & \emph{0.11}\\
    & Random $\boldtheta$ (2) & 27.49 & 0.11\\

